# Chat dialog

Guidance supports chat-based models like ChatGPT and GPT-4 using role tags. These are then converted to the appropriate format for the model (either a JSON API format or special tokens).

In [1]:
import os

from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# This is the name of the model deployed, such as 'gpt-4' or 'gpt-35-turbo
model = os.getenv("AZUREAI_CHAT_MODEL", "Please set the AzureAI model")
# This is the deployment URL, as provided in the Azure AI playground ('view code'):
azure_endpoint = os.getenv("AZUREAI_CHAT_ENDPOINT", "Please set the endpoint")
# The environment variable should be set to the API key from the Azure AI playground:
api_key=os.getenv("AZUREAI_KEY", "Please set API key")
# Alternatively, we can use Entra authentication
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

Preliminaries concluded, we can now create our model:

In [2]:
from guidance import models, gen

azureai_model = models.AzureOpenAIChat(
    model=model,
    azure_endpoint=azure_endpoint,
    # For authentication, use either
    azure_ad_token_provider=token_provider,
    # or
    # api_key=api_key
    temperature = 0.7
)

# Multi-step chat with hidden blocks

We are now going to set up a multistage chat, where we have the chat bot help the use achieve some goal.
The user will only have to specify the goal, and then we will create a chain-of-thought conversation with the bot which will:

1. Ask the bot for a number of suggestions
2. List the pros and cons of each
3. Pick the best suggestion
4. Product a detailed action plan

Our goal is to only show the final result to the user.

Now, let us define our generation function:

In [3]:
import re

import guidance
from guidance import system, user, assistant

@guidance
def plan_for_goal(lm, goal: str):
    # This is a helper function which we will use below
    def parse_best(prosandcons, options):
        best = re.search(r'Best=(\d+)', prosandcons)
        if not best:
            best =  re.search(r'Best.*?(\d+)', 'Best= option is 3')
        if best:
            best = int(best.group(1))
        else:
            best = 0
        return options[best]

    # Some general instruction to the model
    with system():
        lm += "You are a helpful assistant."

    # Simulate a simple request from the user
    # Note that we switch to using 'lm2' here, because these are intermediate steps
    with user():
        lm2 = lm + "I want to " + goal + "\n"
        lm2 += f"Can you please generate one option for how to accomplish this?"
        lm2 += "Please make the option very short, at most one line."

    # Generate several options. Note that this means several sequential generation requests
    n_options = 5
    for i in range(n_options):
        with assistant():
            lm2 += gen(name='options', list_append=True, temperature=1.0, max_tokens=50)

    # Have the user request pros and cons
    with user():
        lm2 += "I want to " + goal + "\n"
        lm2 += "Can you please comment on the pros and cons of each of the following options, and then pick the best option?"
        lm2 += "\n---\n"
        for i, opt in enumerate(lm2['options']):
            lm2 += f"Option {i}: {opt}\n"
        lm2 += "---\n"
        lm2 += "Please discuss each option very briefly (one line for pros, one for cons), and end by saying Best=X, where X is the best option."

    # Get the pros and cons from the model
    with assistant():
        lm2 += gen(name='prosandcons', temperature=0.0, max_tokens=600)

    # The user now extracts the one selected as the best, and asks for a full plan
    # We switch back to 'lm' because this is the final result we want
    with user():
        lm += "I want to " + goal + "\n"
        lm += "Here is my plan:\n"
        lm += parse_best(lm2['prosandcons'], lm2['options'])
        lm += "\n"
        lm += "Please elaborate on this plan, and tell me how to best accomplish it."

    # The plan is generated
    with assistant():
        lm += gen(name='plan', max_tokens=500)

    return lm

Create a plan for the user. Note how the portions which were sent to `lm2` in the function above are not shown in the final result:

In [4]:
results = azureai_model + plan_for_goal(goal="read more books")

We can access the final plan itself:

In [5]:
print(results['plan'])

Setting aside dedicated time each day for reading is a great plan to read more books. Here are some ways you can best accomplish this:

1. Schedule your reading time: Choose a specific time each day that works best for you. It could be in the morning, during lunch break, or before going to bed. Consistency is key, so try to stick to the same schedule every day.

2. Minimize distractions: Create a quiet and comfortable reading environment. Find a cozy spot where you can focus without interruptions. Put away your phone or any other potential distractions that may tempt you away from your reading time.

3. Start with smaller goals: If you're just beginning to develop a reading habit, start with a smaller goal, such as reading for 15 minutes a day. As you become more comfortable and find pleasure in reading, gradually increase the reading time.

4. Choose books you enjoy: Reading becomes more enjoyable when you're interested in the subject matter. Select books that align with your interest

## Asking help from experts

Now, let us ask our chat model to pick some experts in a particular field, and impersonate them to give advice:

In [6]:
@guidance
def run_expert_advice(lm, query: str):
    # Some general instruction to the model
    with system():
        lm += "You are a helpful assistant."

    with user():
        lm += f"""I want a response to the following question:
{query}
Who are 3 world-class experts (past or present) who would be great at answering this?
Please don't answer the question or comment on it yet.
"""

    with assistant():
        lm += gen(name='experts', temperature=0, max_tokens=300)

    with user():
        lm += """Great, now please answer the question as if these experts had collaborated in writing a joint anonymous answer.
In other words, their identity is not revealed, nor is the fact that there is a panel of experts answering the question.
If the experts would disagree, just present their different positions as alternatives in the answer itself (e.g. 'some might argue... others might argue...').
Please start your answer with ANSWER:
"""

    with assistant():
        lm += gen(name='answer', temperature=0, max_tokens=500)

    return lm

In [7]:
mean_life = azureai_model + run_expert_advice("What is the meaning of life?")

In [8]:
more_productive = azureai_model + run_expert_advice('How can I be more productive?')

## Agents

We are now going to define a 'conversation agent.'
This maintains a memory of a conversation, and can generate an appropriate reply, based on the persona it has been given.

In [9]:
class ConversationAgent:
    def __init__(self, chat_model, name: str, instructions: str, context_turns: int = 2):
        self._chat_model = chat_model
        self._name = name
        self._instructions = instructions
        self._my_turns = []
        self._interlocutor_turns = []
        self._went_first = False
        self._context_turns = context_turns

    @property
    def name(self) -> str:
        return self._name
    
    def reply(self, interlocutor_reply: str | None = None) -> str:
        if interlocutor_reply is None:
            self._my_turns = []
            self._interlocutor_turns = []
            self._went_first = True
        else:
            self._interlocutor_turns.append(interlocutor_reply)

        # Get trimmed history
        my_hist = self._my_turns[(1-self._context_turns):]
        interlocutor_hist = self._interlocutor_turns[-self._context_turns:]

        # Set up the system prompt
        curr_model = self._chat_model
        with system():
            curr_model += f"Your name is {self.name}. {self._instructions}"
            if len(interlocutor_hist) == 0:
                curr_model += "Introduce yourself and start the conversation"
            elif len(interlocutor_hist) == 1:
                curr_model += "Introduce yourself before continuing the conversation"

        # Replay the last few turns
        for i in range(len(my_hist)):
            with user():
                curr_model += interlocutor_hist[i]
            with assistant():
                curr_model += my_hist[i]

        if len(interlocutor_hist) > 0:
            with user():
                curr_model += interlocutor_hist[-1]

        with assistant():
            curr_model += gen(name='response', max_tokens=100)

        self._my_turns.append(curr_model['response'])
        return curr_model['response']

We can have two of these agents converse with each other with a _conversation simulator_:

In [10]:
def conversation_simulator(
    bot0: ConversationAgent,
    bot1: ConversationAgent,
    total_turns: int = 5 )-> list[dict[str,str]]:
    conversation_turns = []
    last_reply = None
    for _ in range(total_turns):
        last_reply = bot0.reply(last_reply)
        conversation_turns.append(dict(name=bot0.name, text=last_reply))
        last_reply = bot1.reply(last_reply)
        conversation_turns.append(dict(name=bot1.name, text=last_reply))
    return conversation_turns

Now, let's try generating a conversation:

In [11]:
bot_instructions = """You are taking part in a discussion about bodyline bowling.
Only generate text as yourself and do not prefix your reply with your name.
Keep your answers to a couple of short sentences."""

bradman_bot = ConversationAgent(azureai_model, "Donald Bradman", bot_instructions)
jardine_bot = ConversationAgent(azureai_model, "Douglas Jardine", bot_instructions)

conversation_turns = conversation_simulator(bradman_bot, jardine_bot, total_turns=3)

In [12]:
for turn in conversation_turns:
    print(f"{turn['name']}: {turn['text']}\n")

Donald Bradman: Hi everyone, I'm Donald Bradman, a former Australian cricketer. I'm excited to discuss bodyline bowling and its impact on the game.

Douglas Jardine: Hello Donald, it's a pleasure to have you here. As an avid cricket fan, I'm very interested in discussing the controversial topic of bodyline bowling. It certainly had a significant impact on the game.

Donald Bradman: Thank you for having me here. Bodyline bowling was indeed a controversial tactic that had a significant impact on the game of cricket. It sparked a lot of debate and changed the way the game was played.

Douglas Jardine: Absolutely, Donald. Bodyline bowling brought about a shift in tactics, forcing batsmen to adapt and altering the balance between bat and ball. Its impact on the game is undeniable.

Donald Bradman: You're absolutely right. Bodyline bowling forced batsmen to develop new techniques and strategies to counter it, and it certainly changed the dynamics of the game, making it more challenging for t

<hr style="height: 1px; opacity: 0.5; border: none; background: #cccccc;">
<div style="text-align: center; opacity: 0.5">Have an idea for more helpful examples? Pull requests that add to this documentation notebook are encouraged!</div>